In [17]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/NASH


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/NASH


In [18]:
import warnings
import io
import os
import csv
import time
import requests
import subprocess
from requests.auth import HTTPBasicAuth
from typing import Optional, List
import tempfile
import io
import os
import csv
import time
import shlex
import torch
import torchaudio
import subprocess
import torch.nn.functional as F
import requests
from typing import Optional, List
import os
import io
import csv
import time
import shlex
import tempfile
import subprocess
from typing import List, Optional, Tuple

import torch
import torchaudio
import torch.nn.functional as F

from urllib.parse import urljoin
from requests.auth import HTTPBasicAuth
from bs4 import BeautifulSoup

from aed_models import *


warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [19]:
#pip install paramiko
import paramiko

In [20]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# === PARAMETRI MODELLO ===
sample_rate = 16000
duration = 1
seg_len = sample_rate * duration
threshold = 0.0020

In [21]:
import os
import io
import shlex
import csv
import time
import tempfile
from typing import List, Tuple, Optional

import torch
import torch.nn.functional as F
import torchaudio
import paramiko

# ==== CONFIG SSH REMOTA ====
REMOTE_HOST = "151.14.46.173"
REMOTE_USER = "milone"
SSH_PORT = 2200
SSH_KEY = "id_rsa"  # percorso alla tua chiave privata
UUID = "8c69c0b3-ae12-4552-9b11-0aaa0304a06d"
REMOTE_BASE = f"/mnt/BB-S_STORAGE/{UUID}/recordings"

# ==== CONFIG AUDIO ====
sample_rate = 16000
seg_len = sample_rate * 1  # 1 secondo di default
threshold = 0.01

# ================== FUNZIONI AUDIO ==================
def load_audio(file_like, target_length: int = seg_len) -> Tuple[torch.Tensor, int]:
    waveform, sr = torchaudio.load(file_like)
    if sr != sample_rate:
        resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=sample_rate)
        waveform = resampler(waveform)

    if waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0, keepdim=True)

    original_length = waveform.shape[1]

    if original_length > target_length:
        waveform = waveform[:, :target_length]
        original_length = target_length
    else:
        pad_len = target_length - original_length
        waveform = F.pad(waveform, (0, pad_len))

    return waveform.squeeze(0), original_length


def run_inference(model, audio_tensor: torch.Tensor, original_length: int, threshold: float = threshold):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    if hasattr(model, "transform_tf"):
        model.transform_tf = model.transform_tf.to(device)

    audio_tensor = audio_tensor.unsqueeze(0).to(device)
    feature_vector = model.preprocessing(audio_tensor)

    hop_length = getattr(getattr(model, "transform_tf", None), "hop_length", 512)
    real_t_bins = 1 + (original_length // hop_length)
    frames_field = getattr(model, "frames", 1)
    real_vector_array_size = max(1, real_t_bins - frames_field + 1)

    with torch.no_grad():
        encoded = model.encoder(feature_vector)
        bottleneck = model.bottleneck(encoded)
        reconstructed = model.decoder(bottleneck)

    feature_vector = feature_vector[:, :real_vector_array_size, :]
    reconstructed = reconstructed[:, :real_vector_array_size, :]

    loss = F.mse_loss(reconstructed, feature_vector).item()
    return feature_vector.cpu(), reconstructed.cpu(), loss


# ================== SSH HELPERS (paramiko) ==================
def _open_ssh():
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(REMOTE_HOST, port=SSH_PORT, username=REMOTE_USER, key_filename=SSH_KEY)
    return ssh


def _ssh_exec(remote_cmd: str, check: bool = True):
    ssh = _open_ssh()
    stdin, stdout, stderr = ssh.exec_command(remote_cmd)
    out = stdout.read().decode("utf-8", errors="ignore")
    err = stderr.read().decode("utf-8", errors="ignore")
    code = stdout.channel.recv_exit_status()
    ssh.close()
    if check and code != 0:
        raise RuntimeError(f"SSH error ({code}): {err.strip()}")
    return out


def _ssh_exec_bin(remote_cmd: str, check: bool = True):
    ssh = _open_ssh()
    stdin, stdout, stderr = ssh.exec_command(remote_cmd)
    out = stdout.read()
    err = stderr.read().decode("utf-8", errors="ignore")
    code = stdout.channel.recv_exit_status()
    ssh.close()
    if check and code != 0:
        raise RuntimeError(f"SSH error ({code}): {err.strip()}")
    return out


def _scp_to_temp(remote_path: str) -> str:
    ssh = paramiko.Transport((REMOTE_HOST, SSH_PORT))
    ssh.connect(username=REMOTE_USER, pkey=paramiko.RSAKey.from_private_key_file(SSH_KEY))
    sftp = paramiko.SFTPClient.from_transport(ssh)
    fd, tmp_path = tempfile.mkstemp(suffix=os.path.splitext(remote_path)[1])
    os.close(fd)
    sftp.get(remote_path, tmp_path)
    sftp.close()
    ssh.close()
    return tmp_path


def remote_delete_first_line(list_path: str) -> None:
    remote_cmd = f"sed -i '1d' {shlex.quote(list_path)}"
    _ssh_exec(remote_cmd, check=True)


def remote_head_first_line(list_path: str) -> str:
    remote_cmd = f"head -n 1 {shlex.quote(list_path)}"
    out = _ssh_exec(remote_cmd, check=False).strip().replace("\r", "")
    return out


def debug_queue_path(device: str):
    list_path = f"{REMOTE_BASE}/BB-AI_{device}_LIST.txt"
    for cmd in [
        f"ls -l {shlex.quote(list_path)}",
        f"wc -l {shlex.quote(list_path)}",
        f"nl -ba {shlex.quote(list_path)} | sed -n '1,3p' | sed -e 's/\\r/$/g'"
    ]:
        print(f"[DBG] ssh> {cmd}")
        out = _ssh_exec(cmd, check=False)
        print(out or "(no output)")


# ================== ANALISI FILE REMOTI ==================
def analyze_remote_file(remote_path: str, model) -> float:
    try:
        data = _ssh_exec_bin(f"cat {shlex.quote(remote_path)}", check=True)
        if not data:
            raise RuntimeError("File vuoto o non leggibile")
        audio_data = io.BytesIO(data)
        waveform, real_samples = load_audio(audio_data)
        _, _, loss = run_inference(model, waveform, real_samples)
        return loss
    except Exception:
        tmp_path = None
        try:
            tmp_path = _scp_to_temp(remote_path)
            waveform, sr = torchaudio.load(tmp_path)
            if sr != sample_rate:
                resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=sample_rate)
                waveform = resampler(waveform)
            if waveform.shape[0] > 1:
                waveform = waveform.mean(dim=0, keepdim=True)
            original_length = waveform.shape[1]
            if original_length > seg_len:
                waveform = waveform[:, :seg_len]
                original_length = seg_len
            else:
                pad_len = seg_len - original_length
                waveform = F.pad(waveform, (0, pad_len))
            waveform = waveform.squeeze(0)
            _, _, loss = run_inference(model, waveform, original_length)
            return loss
        finally:
            if tmp_path and os.path.exists(tmp_path):
                os.remove(tmp_path)


# ================== QUEUE: FULL SSH ==================
def list_remote_audio_files(giorno: str, device: str) -> List[str]:
    base_dir = f"{REMOTE_BASE}/{giorno}/device/{device}"

    find_cmd = (
    f"test -d {shlex.quote(base_dir)} && "
    f"find {shlex.quote(base_dir)} -type f ( -iname '*.wav' -o -iname '*.flac' ) || true")

    out = _ssh_exec(find_cmd, check=False)
    files = [ln.strip() for ln in (out or "").splitlines() if ln.strip()]
    return files


def _parse_queue_line_to_base_url(line: str, uuid: str = UUID) -> Optional[str]:
    line = (line or "").strip()
    if not line:
        return None
    if line.startswith(("http://", "https://")):
        return line
    parts = [p.strip() for p in line.split(",")]
    if len(parts) >= 2:
        wav_path = parts[0]
        giorno = parts[1]
        device = None
        try:
            if wav_path and not wav_path.startswith("/"):
                wav_path = "/" + wav_path
            segs = [s for s in wav_path.split("/") if s]
            if "device" in segs:
                idx = segs.index("device")
                if idx + 1 < len(segs):
                    device = segs[idx + 1]
        except Exception:
            device = None
        if giorno and device:
            return f"https://lys-ai.it/recordings/{uuid}/recordings/{giorno}/FLAC/device/{device}"
    return None


def consume_queue_full_ssh(device: str,
                           model,
                           model_type: str,
                           csv_name: str,
                           poll_every_seconds: int = 10):
    if not device:
        raise ValueError("DEVICE non può essere vuoto")

    list_path = f"{REMOTE_BASE}/BB-AI_{device}_LIST.txt"
    print(f"📖 [QUEUE][SSH] In ascolto su: {list_path}")

    out_dir = f"esiti_{csv_name}"
    os.makedirs(out_dir, exist_ok=True)

    while True:
        line = remote_head_first_line(list_path)
        if not line:
            print(f"[QUEUE][SSH] Nessuna riga trovata, attendo... (file: {list_path})")
            time.sleep(poll_every_seconds)
            continue

        print(f"👉 Riga da processare (raw): {line}")
        parts = [p.strip() for p in line.split(",")]
        print(f"[DBG] Split CSV -> {parts}")

        anomalies: List[Tuple[str, str, float]] = []
        giorno_for_csv = "unknown"
        ok = False

        try:
            if len(parts) >= 2 and parts[0] and (parts[0].startswith("/") or parts[0].startswith("mnt")):
                wav_path = parts[0]
                giorno_for_csv = parts[1] or "unknown"
                if not wav_path.startswith("/"):
                    wav_path = "/" + wav_path
                print(f"[SSH] Analisi singolo file: {wav_path}")
                try:
                    loss = analyze_remote_file(wav_path, model)
                    status = "Anomalo" if loss > threshold else "Normale"
                    print(f" --> {status} (MSE: {loss:.6f})")
                    if status == "Anomalo":
                        anomalies.append((wav_path, status, loss))
                    ok = True
                except Exception as e:
                    print(f"❌ Errore su {wav_path}: {e}")
                    ok = False

            else:
                url = _parse_queue_line_to_base_url(line)
                if not url:
                    print(f"[SSH] Riga non interpretabile: {line!r}")
                    ok = False
                else:
                    try:
                        tail = url.split("/recordings/")[1]
                        giorno_for_csv = tail.split("/")[0]
                        device_name = tail.split("/device/")[1].split("/")[0]
                    except Exception:
                        print(f"[SSH] Impossibile estrarre giorno/device da URL: {url}")
                        ok = False
                    else:
                        paths = list_remote_audio_files(giorno_for_csv, device_name)
                        if not paths:
                            print(f"[SSH] Nessun file trovato per {giorno_for_csv} / {device_name}")
                            ok = True
                        else:
                            print(f"[SSH] Trovati {len(paths)} file. Avvio analisi...")
                            ok = True
                            for rp in paths:
                                try:
                                    loss = analyze_remote_file(rp, model)
                                    status = "Anomalo" if loss > threshold else "Normale"
                                    print(f"  {os.path.basename(rp)} -> {status} (MSE: {loss:.6f})")
                                    if status == "Anomalo":
                                        anomalies.append((rp, status, loss))
                                except Exception as e:
                                    print(f"  ❌ Errore su {rp}: {e}")
                                    ok = False

            csv_filename = os.path.join(out_dir, f"{model_type}_esito_{giorno_for_csv}.csv")
            with open(csv_filename, mode='w', newline='') as f:
                w = csv.writer(f)
                w.writerow(["path", "status", "loss"])
                for pth, st, ls in anomalies:
                    w.writerow([pth, st, f"{ls:.6f}"])
            print(f"[SSH] Salvate {len(anomalies)} anomalie in '{csv_filename}'")

            prepend_metadata_to_csv(
                csv_filename=csv_filename,
                modello="NS V1.01S_V5",
                tipo_file="WAV/FLAC",
                durata_segmento="1",
                verificato1="0",
                verificato2="1"
            )

            if ok:
                try:
                    remote_delete_first_line(list_path)
                except Exception as e:
                    print(f"⚠️  Impossibile cancellare la riga su {list_path}: {e}")

        except Exception as outer:
            print(f"❌ Errore generale sulla riga: {outer}")

        time.sleep(poll_every_seconds)


# ================== UTILITY EXTRA ==================
def prepend_metadata_to_csv(csv_filename: str,
                            modello: str,
                            tipo_file: str,
                            durata_segmento: str,
                            verificato1: str,
                            verificato2: str) -> None:
    header = ["modello", "tipo_file", "durata_segmento", "verificato1", "verificato2"]
    metadata = [modello, tipo_file, durata_segmento, verificato1, verificato2]

    try:
        with open(csv_filename, mode='r') as original:
            original_lines = original.readlines()

        with open(csv_filename, mode='w', newline='') as modified:
            writer = csv.writer(modified)
            writer.writerow(header)
            writer.writerow(metadata)
            modified.writelines(original_lines)

        print(f"✅ Meta-intestazione inserita in '{csv_filename}'")
    except Exception as e:
        print(f"❌ Errore durante l'inserimento della meta-intestazione in {csv_filename}: {e}")


def one_shot_trim_first_line(device: str):
    if not device:
        raise ValueError("DEVICE non può essere vuoto")
    list_path = f"{REMOTE_BASE}/BB-AI_{device}_LIST.txt"
    remote_delete_first_line(list_path)
    print(f"Prima riga rimossa da: {list_path}")


In [ ]:
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model_path = "results_1s_v5/best_model_1s_v5.pth"

    # Istanzia il modello e carica i pesi
    model = NS_1()
    model.load_state_dict(torch.load(model_path, map_location=device))

    model_type = "1s"
    csv_name = "2308_v5"

    # Avvio del consumer SSH
    consume_queue_full_ssh(
        device="RSP1-MIC1",   # Nome del device che vuoi monitorare
        model=model,          # <--- qui passi l'istanza, non la classe
        model_type=model_type,
        csv_name=csv_name,
        poll_every_seconds=5
    )


📖 [QUEUE][SSH] In ascolto su: /mnt/BB-S_STORAGE/8c69c0b3-ae12-4552-9b11-0aaa0304a06d/recordings/BB-AI_RSP1-MIC1_LIST.txt
👉 Riga da processare (raw): /mnt/BB-S_STORAGE/8c69c0b3-ae12-4552-9b11-0aaa0304a06d/recordings/2025-246/device/RSP1-MIC1/RSP1-MIC1_20250903_230929.wav,2025-246,/mnt/BB-S_STORAGE/8c69c0b3-ae12-4552-9b11-0aaa0304a06d/output/2025-246,/mnt/BB-S_STORAGE/8c69c0b3-ae12-4552-9b11-0aaa0304a06d/output/2025-246/device/RSP1-MIC1
[DBG] Split CSV -> ['/mnt/BB-S_STORAGE/8c69c0b3-ae12-4552-9b11-0aaa0304a06d/recordings/2025-246/device/RSP1-MIC1/RSP1-MIC1_20250903_230929.wav', '2025-246', '/mnt/BB-S_STORAGE/8c69c0b3-ae12-4552-9b11-0aaa0304a06d/output/2025-246', '/mnt/BB-S_STORAGE/8c69c0b3-ae12-4552-9b11-0aaa0304a06d/output/2025-246/device/RSP1-MIC1']
[SSH] Analisi singolo file: /mnt/BB-S_STORAGE/8c69c0b3-ae12-4552-9b11-0aaa0304a06d/recordings/2025-246/device/RSP1-MIC1/RSP1-MIC1_20250903_230929.wav
 --> Normale (MSE: 0.001508)
[SSH] Salvate 0 anomalie in 'esiti_2308_v5/1s_esito_2025-246